In [1]:
sc.master

'local[*]'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
sc = spark.sparkContext

path = "/CA2/polairty_df.csv"

df = spark.read.csv(path, header = True, inferSchema = True)

In [4]:
df.tail(5)

[Row(_c0=1632415, date='2009-04-18', test='radiodisney HEEECK YEAH', negative=0, neutral=1, positive=0),
 Row(_c0=1632416, date='2009-04-18', test='blast Bachelorette Party Watch photo video FB multiply', negative=0, neutral=1, positive=0),
 Row(_c0=1632417, date='2009-04-18', test='Hey Sam old wa Dougie started playing bass xx', negative=1, neutral=0, positive=0),
 Row(_c0=1632418, date='2009-04-18', test='hello', negative=0, neutral=1, positive=0),
 Row(_c0=1632419, date='2009-04-18', test='im excited see tonight time planning going', negative=0, neutral=0, positive=1)]

In [5]:
df.show()

+---+----------+--------------------+--------+-------+--------+
|_c0|      date|                test|negative|neutral|positive|
+---+----------+--------------------+--------+-------+--------+
|  0|2009-04-06|Awww thats bummer...|       0|      0|       1|
|  1|2009-04-06|upset cant update...|       0|      1|       0|
|  2|2009-04-06|dived many time b...|       0|      0|       1|
|  3|2009-04-06|whole body feel i...|       0|      0|       1|
|  4|2009-04-06|behaving im mad c...|       1|      0|       0|
|  5|2009-04-06|          whole crew|       0|      0|       1|
|  6|2009-04-06|            Need hug|       0|      1|       0|
|  7|2009-04-06|hey long time see...|       0|      0|       1|
|  8|2009-04-06|          nope didnt|       0|      1|       0|
|  9|2009-04-06|           que muera|       0|      1|       0|
| 10|2009-04-06|spring break plai...|       1|      0|       0|
| 11|2009-04-06|       repierced ear|       0|      1|       0|
| 12|2009-04-06|couldnt bear watc...|   

2023-11-07 00:24:51,805 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, test, negative, neutral, positive
 Schema: _c0, date, test, negative, neutral, positive
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/CA2/polairty_df.csv


In [6]:
from pyspark.sql.functions import array

In [7]:
df = df.withColumn("array", array("test"))

In [8]:
df.show()

2023-11-07 00:24:52,929 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, test, negative, neutral, positive
 Schema: _c0, date, test, negative, neutral, positive
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/CA2/polairty_df.csv


+---+----------+--------------------+--------+-------+--------+--------------------+
|_c0|      date|                test|negative|neutral|positive|               array|
+---+----------+--------------------+--------+-------+--------+--------------------+
|  0|2009-04-06|Awww thats bummer...|       0|      0|       1|[Awww thats bumme...|
|  1|2009-04-06|upset cant update...|       0|      1|       0|[upset cant updat...|
|  2|2009-04-06|dived many time b...|       0|      0|       1|[dived many time ...|
|  3|2009-04-06|whole body feel i...|       0|      0|       1|[whole body feel ...|
|  4|2009-04-06|behaving im mad c...|       1|      0|       0|[behaving im mad ...|
|  5|2009-04-06|          whole crew|       0|      0|       1|        [whole crew]|
|  6|2009-04-06|            Need hug|       0|      1|       0|          [Need hug]|
|  7|2009-04-06|hey long time see...|       0|      0|       1|[hey long time se...|
|  8|2009-04-06|          nope didnt|       0|      1|       0|  

In [9]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import monotonically_increasing_id

In [11]:
cv = CountVectorizer(inputCol="array",outputCol="rawFeatures",minDF=5, maxDF=0.9)
cv_model = cv.fit(df)
featurized_data = cv_model.transform(df)

In [12]:
featurized_data.show()

+---+----------+--------------------+--------+-------+--------+--------------------+-------------------+
|_c0|      date|                test|negative|neutral|positive|               array|        rawFeatures|
+---+----------+--------------------+--------+-------+--------+--------------------+-------------------+
|  0|2009-04-06|Awww thats bummer...|       0|      0|       1|[Awww thats bumme...|       (4129,[],[])|
|  1|2009-04-06|upset cant update...|       0|      1|       0|[upset cant updat...|       (4129,[],[])|
|  2|2009-04-06|dived many time b...|       0|      0|       1|[dived many time ...|       (4129,[],[])|
|  3|2009-04-06|whole body feel i...|       0|      0|       1|[whole body feel ...|       (4129,[],[])|
|  4|2009-04-06|behaving im mad c...|       1|      0|       0|[behaving im mad ...|       (4129,[],[])|
|  5|2009-04-06|          whole crew|       0|      0|       1|        [whole crew]|       (4129,[],[])|
|  6|2009-04-06|            Need hug|       0|      1| 

2023-11-07 00:26:08,308 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, test, negative, neutral, positive
 Schema: _c0, date, test, negative, neutral, positive
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/CA2/polairty_df.csv


In [13]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(featurized_data)
tfidf_data = idf_model.transform(featurized_data)

In [15]:
tfidf_data.select("date","features", "negative", "positive", "neutral").show()

+----------+--------------------+--------+--------+-------+
|      date|            features|negative|positive|neutral|
+----------+--------------------+--------+--------+-------+
|2009-04-06|        (4129,[],[])|       0|       1|      0|
|2009-04-06|        (4129,[],[])|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       0|       1|      0|
|2009-04-06|        (4129,[],[])|       0|       1|      0|
|2009-04-06|        (4129,[],[])|       1|       0|      0|
|2009-04-06|        (4129,[],[])|       0|       1|      0|
|2009-04-06|        (4129,[],[])|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       0|       1|      0|
|2009-04-06|(4129,[3645],[12....|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       1|       0|      0|
|2009-04-06|        (4129,[],[])|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       0|       0|      1|
|2009-04-06|        (4129,[],[])|       

In [ ]:
#pip install pandas

In [16]:
import pandas as pd

In [17]:
tfidf_data.select("date", "test", "features", "negative", "positive", "neutral").withColumn("date", monotonically_increasing_id()).toPandas().to_csv('tfidf_data.csv', index=False)